In [5]:
import pinax
import pandas as pd
import numpy as np

In [3]:
## Extract Table from CSV

# Parameters
file_path = '../examples/data/SampleRoadAcceleration.csv'
table_marker = 'Line'
target_header_num = 3 # Should be 1 or more
total_headers_num = 4
limit_lines_search = 300 

# Loop
flag_marker = False
with open(file_path) as f:
    for i, line in enumerate(f):
        if (table_marker in line) and not flag_marker:
            # Assignation
            first_header_row = i
            target_header_row = i + target_header_num - 1
            table_row = i + total_headers_num
            flag_marker = True

        if flag_marker:
            if i == target_header_row:
                # Header row extraction
                columns = [col.strip() for col in line.split(',')]
                break

        if i > limit_lines_search:
            raise ValueError(f"Marker '{table_marker}' not found within the first {limit_lines_search} lines of the file.")

# DataFrame
df = pd.read_csv(file_path, skiprows=table_row, header=None, names=columns)

In [4]:
df

,Line,Abs Time(Sec),Rel Time (Sec),Status,Er,Tx,Description,Network,Node,Arb ID,...,B2,B3,B4,B5,B6,B7,B8,Value,Trigger,Signals
0,1,0.574781,0.000000,603979776,F,F,DW CAN 07 $206,DW CAN 07,NaN,206,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,0.575054,0.000273,603979776,F,F,DW CAN 02 $206,DW CAN 02,NaN,206,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,0.575097,0.000043,603979776,F,F,DW CAN 01 $42A,DW CAN 01,NaN,42A,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,0.575231,0.000135,603979780,F,F,DW CAN 07 $18DA44F2,DW CAN 07,NaN,18DA44F2,...,22,A0,0A,00,00,00,00,NaN,NaN,NaN
4,5,0.575649,0.000418,67108864,F,F,DW CAN 05 $354,DW CAN 05,NaN,354,...,00,04,00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
625701,625702,89.509929,0.000062,67108868,F,F,DW CAN 03 $18DAF198,DW CAN 03,NaN,18DAF198,...,62,02,73,00,00,00,00,NaN,NaN,NaN
625702,625703,89.510180,0.000251,603979776,F,F,DW CAN 01 $106,DW CAN 01,NaN,106,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
625703,625704,89.510562,0.000382,603979776,F,F,DW CAN 01 $1DC,DW CAN 01,NaN,1DC,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
625704,625705,89.510599,0.000037,603979776,F,F,DW CAN 02 $106,DW CAN 02,NaN,106,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
# Efficient, vectorized payload extraction for up to 64 bytes

# Parameters
extended_payload_cols = [f'B{i}' for i in range(1, 65)]
current_payload_cols = [f'B{i}' for i in range(1, 9)]

# Identify rows with long payloads (space in B1)
long_payload_mask = df['B1'].astype(str).str.contains(' ')

# Split long payloads into bytes (vectorized) and rename columns to B1, B2, ..., B64
long_payload_bytes = (
    df.loc[long_payload_mask, 'B1']
    .str.strip()
    .str.split(' ', expand=True)
)
long_payload_bytes.columns = extended_payload_cols[:long_payload_bytes.shape[1]]

# For short payloads, stack B1-B8 columns as strings (vectorized)
short_payload_bytes = (
    df.loc[~long_payload_mask, current_payload_cols]
)

# Pad both to 64 columns
def pad_to_64(df_bytes):
    cols = [f'B{i}' for i in range(1, 65)]
    df_bytes = df_bytes.reindex(columns=cols, fill_value=np.nan)
    return df_bytes

long_payload_bytes = pad_to_64(long_payload_bytes)
short_payload_bytes = pad_to_64(short_payload_bytes)

# Combine back into one DataFrame, preserving original order
payload_df = pd.concat([long_payload_bytes, short_payload_bytes]).sort_index()

# Reorganize columns to match the original payload structure
first_payload_idx = df.columns.get_loc(current_payload_cols[0])
new_col_order = (
    list(df.columns[:first_payload_idx]) +
    extended_payload_cols +
    list(df.columns[first_payload_idx + len(current_payload_cols):])
)
# Create the extended dataframe
df_extended = pd.concat([df.drop(columns=current_payload_cols), payload_df], axis=1)
df_extended = df_extended[new_col_order]



In [ ]:
from pinax.amtl_tools.vspy_tools import vspy_buffer_csv_to_df, vspy_buffer_df_extend_bytes


In [ ]:
df = vspy_buffer_csv_to_df('../examples/data/SampleRoadAcceleration.csv')